In [17]:
import pandas as pd
import json
import numpy
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine

In [2]:
df = pd.read_csv('../../data/CDR/sms-call-internet-mi-2013-11-01.txt', delimiter='\t',
                 header=None) 

In [3]:
file = '../../data/GeoJSON/milano-grid.geojson'
with open(file) as f:
    grid = pd.read_json(f, typ='Series')

In [9]:
grid

crs         {'properties': {'name': 'urn:ogc:def:crs:EPSG:...
features    [{'id': 0, 'properties': {'cellId': 1}, 'geome...
type                                        FeatureCollection
dtype: object

In [4]:
df.columns = ["cellID", "time", "countryCode", "smsIn", "smsOut", "callIn", "callOut", "internet"]

In [47]:
for cell in grid["features"]:
    if 'sms_in' not in cell['properties']:
        cell['properties']['sms_in'] = df[(df["cellID"] == cell['properties']['cellId']) & (df["countryCode"] != 0)]['smsIn'].sum()
    if 'sms_out' not in cell['properties']:
        cell['properties']['sms_out'] = df[(df["cellID"] == cell['properties']['cellId']) & (df["countryCode"] != 0)]['smsOut'].sum()
    if 'call_in' not in cell['properties']:
        cell['properties']['call_in'] = df[(df["cellID"] == cell['properties']['cellId']) & (df["countryCode"] != 0)]['callIn'].sum()
    if 'call_out' not in cell['properties']:
        cell['properties']['call_out'] = df[(df["cellID"] == cell['properties']['cellId']) & (df["countryCode"] != 0)]['callOut'].sum()
    if 'internet' not in cell['properties']:
        cell['properties']['internet'] = df[(df["cellID"] == cell['properties']['cellId']) & (df["countryCode"] != 0)]['internet'].sum()    

KeyboardInterrupt: 

In [5]:
gg = json_normalize(grid['features'])

In [6]:
df.head()

,cellID,time,countryCode,smsIn,smsOut,callIn,callOut,internet
0,1,1383260400000,0,0.081363,NaN,NaN,NaN,NaN
1,1,1383260400000,39,0.141864,0.156787,0.160938,0.052275,11.028366
2,1,1383261000000,0,0.136588,NaN,NaN,0.027300,NaN
3,1,1383261000000,33,NaN,NaN,NaN,NaN,0.026137
4,1,1383261000000,39,0.278452,0.119926,0.188777,0.133637,11.100963


In [7]:
gg.head()

,geometry.coordinates,geometry.type,id,properties.cellId,type
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature


,geometry.coordinates,geometry.type,id,properties.cellId,type,sms_in
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature,1.23
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature,1.23
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature,1.23
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature,1.23
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature,1.23


In [8]:
agg_df = df[df['countryCode']!=0].groupby('cellID').agg({
        'cellID': 'first',
        'time': 'first',
        'smsIn': 'sum',
        'smsOut': 'sum',
        'callIn': 'sum',
        'callOut': 'sum',
        'internet': 'sum'
    })

In [9]:
agg_df.head()

,cellID,callIn,callOut,time,smsIn,smsOut,internet
cellID,,,,,,,
1,1,40.947090,47.574988,1383260400000,57.888593,45.827771,1507.048349
2,2,41.578010,48.467323,1383260400000,58.672131,46.423165,1512.859757
3,3,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
4,4,39.119618,44.990324,1383260400000,55.619062,44.103200,1490.215500
5,5,36.720678,42.165447,1383260400000,51.638382,40.994705,1356.849530


In [70]:
df.head()

,cellID,time,countryCode,smsIn,smsOut,callIn,callOut,internet
0,1,1383260400000,0,0.081363,NaN,NaN,NaN,NaN
1,1,1383260400000,39,0.141864,0.156787,0.160938,0.052275,11.028366
2,1,1383261000000,0,0.136588,NaN,NaN,0.027300,NaN
3,1,1383261000000,33,NaN,NaN,NaN,NaN,0.026137
4,1,1383261000000,39,0.278452,0.119926,0.188777,0.133637,11.100963


In [10]:
merged = pd.merge(left=gg, right=agg_df, left_on='properties.cellId', right_on='cellID')

In [11]:
merged.head()

,geometry.coordinates,geometry.type,id,properties.cellId,type,cellID,callIn,callOut,time,smsIn,smsOut,internet
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature,1,40.947090,47.574988,1383260400000,57.888593,45.827771,1507.048349
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature,2,41.578010,48.467323,1383260400000,58.672131,46.423165,1512.859757
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature,3,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature,4,39.119618,44.990324,1383260400000,55.619062,44.103200,1490.215500
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature,5,36.720678,42.165447,1383260400000,51.638382,40.994705,1356.849530


In [74]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'crs':{'type':'name','properties':{}}, 'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature', 'properties':{},'id': row['id'], 'geometry':{'type':'Polygon','coordinates':[]}}
        feature['geometry']['coordinates'] = row['geometry.coordinates']
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [75]:
cols = ['cellID','smsIn','smsOut','callIn','callOut','internet']
geojson = df_to_geojson(merged, cols)

In [80]:
output_filename = 'joined.geojson'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2)

In [16]:
merged.sort_values(by = 'cellID')

,geometry.coordinates,geometry.type,id,properties.cellId,type,cellID,callIn,callOut,time,smsIn,smsOut,internet
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature,1,40.947090,47.574988,1383260400000,57.888593,45.827771,1507.048349
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature,2,41.578010,48.467323,1383260400000,58.672131,46.423165,1512.859757
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature,3,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature,4,39.119618,44.990324,1383260400000,55.619062,44.103200,1490.215500
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature,5,36.720678,42.165447,1383260400000,51.638382,40.994705,1356.849530
5,"[[[9.026493248013145, 45.35879881990051], [9.0...",Polygon,5,6,Feature,6,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
6,"[[[9.029493687770275, 45.35879808454441], [9.0...",Polygon,6,7,Feature,7,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
7,"[[[9.032494127405446, 45.35879727037025], [9.0...",Polygon,7,8,Feature,8,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
8,"[[[9.035494566906245, 45.358796377378034], [9....",Polygon,8,9,Feature,9,42.249600,49.417178,1383260400000,59.506176,47.056939,1519.045771
9,"[[[9.038495006260266, 45.35879540556776], [9.0...",Polygon,9,10,Feature,10,33.122223,39.176358,1383260400000,43.466583,34.475793,912.280072
